# Simple Entry Removal Algorithm 2 (SER 2)

In [1]:
import numpy as np
import datetime
import random
import pandas as pd
import json

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [2]:
def add_up_a_decomposition(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=int)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# The SER 2 algorithm

In [3]:
# This function is used in the SER 2 function.
# residue_matrix_R is a np.ndarray.
# chosen_entry_pos_each_col is a 1-by-PBN_col_sum np.ndarray.
# PBN_col_num is the number of columns present in residue_matrix_R

def choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num):
    col_counter = 0
    chosen_entry_xi = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
    
    for col_counter in range(PBN_col_num):
        current_entry = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
        if current_entry < chosen_entry_xi:
            chosen_entry_xi = current_entry
    
    return chosen_entry_xi

In [4]:
def SER_2(PBN_matrix_P):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4

    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

# "Truncated Floating-Point Matrix" Experiment

In [5]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

## $8 \times 8$ 5300-TPMs

In [6]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './8-by-8 (trunc)/dim8_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_2(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_2)/dim8_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_2)/dim8_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 17:35:24.712941

iteration 1
Time now is: 2025-04-12 17:35:24.717938

iteration 2
Time now is: 2025-04-12 17:35:24.750848

iteration 3
Time now is: 2025-04-12 17:35:24.755186

iteration 4
Time now is: 2025-04-12 17:35:24.758187

iteration 5
Time now is: 2025-04-12 17:35:24.761692

iteration 6
Time now is: 2025-04-12 17:35:24.764861

iteration 7
Time now is: 2025-04-12 17:35:24.769882

iteration 8
Time now is: 2025-04-12 17:35:24.773386

iteration 9
Time now is: 2025-04-12 17:35:24.778389



## $16 \times 16$ 5300-TPMs

In [10]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './16-by-16 (trunc)/dim16_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_2(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_2)/dim16_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_2)/dim16_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 17:41:12.073052

iteration 1
Time now is: 2025-04-12 17:41:12.080790

iteration 2
Time now is: 2025-04-12 17:41:12.086960

iteration 3
Time now is: 2025-04-12 17:41:12.096104

iteration 4
Time now is: 2025-04-12 17:41:12.110346

iteration 5
Time now is: 2025-04-12 17:41:12.118121

iteration 6
Time now is: 2025-04-12 17:41:12.126077

iteration 7
Time now is: 2025-04-12 17:41:12.134790

iteration 8
Time now is: 2025-04-12 17:41:12.142444

iteration 9
Time now is: 2025-04-12 17:41:12.150899



## $32 \times 32$ 5300-TPMs

In [14]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './32-by-32 (trunc)/dim32_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_2(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_2)/dim32_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_2)/dim32_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 17:45:42.494645

iteration 1
Time now is: 2025-04-12 17:45:42.512459

iteration 2
Time now is: 2025-04-12 17:45:42.539348

iteration 3
Time now is: 2025-04-12 17:45:42.555650

iteration 4
Time now is: 2025-04-12 17:45:42.570192

iteration 5
Time now is: 2025-04-12 17:45:42.587364

iteration 6
Time now is: 2025-04-12 17:45:42.604222

iteration 7
Time now is: 2025-04-12 17:45:42.624418

iteration 8
Time now is: 2025-04-12 17:45:42.639919

iteration 9
Time now is: 2025-04-12 17:45:42.656159



## $64 \times 64$ 5300-TPMs

In [20]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './64-by-64 (trunc)/dim64_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_2(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_2)/dim64_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_2)/dim64_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 17:49:42.425026

iteration 1
Time now is: 2025-04-12 17:49:42.462649

iteration 2
Time now is: 2025-04-12 17:49:42.493002

iteration 3
Time now is: 2025-04-12 17:49:42.550500

iteration 4
Time now is: 2025-04-12 17:49:42.654454

iteration 5
Time now is: 2025-04-12 17:49:42.697124

iteration 6
Time now is: 2025-04-12 17:49:42.732238

iteration 7
Time now is: 2025-04-12 17:49:42.763118

iteration 8
Time now is: 2025-04-12 17:49:42.808345

iteration 9
Time now is: 2025-04-12 17:49:42.856900

